## 13번 모델
- ray 
- fillna
- baseline prohet 
    - changepoint_prior_scale = 0.05 -> T + P 에서만 0.01
    - seasonality_prior_scale = 10 디폴트
- results- ray 

---
- pH에선 로스 증가  
- COD 에서도 로스 감소      
- SS 에서 로스 약간 감소 0.001       
- N에서 로스 약간 증가    
- P에서 로스 약간 증가    


---
## 파라미터 설명 참조 
https://velog.io/@choijhyeok/Prophet-1-parameter-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0

In [1]:
import ray

In [3]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet
import pickle
import copy
import time
from functools import *
from tqdm.auto import tqdm

In [4]:
%%time
''' 베이스라인 코드에서 json으로 DF만듬. 이미 만들어진 데이터를 로드 '''
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb')) # 모든 측정 지점의 피쳐 값들 
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb')) # 측정 지점 리스트
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T'] # 피쳐 리스트

CPU times: user 755 ms, sys: 124 ms, total: 879 ms
Wall time: 879 ms


---
## 데이터 전처리 

In [5]:
%%time
''' fillna : 단순 missing value 채움'''
for plc in plc_lst:
    df = df_dic[plc]
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')    
    df_dic[plc] = df.dropna()
    
    if df_dic[plc].isna().any().any():
        print(plc)

CPU times: user 2.22 s, sys: 12.3 ms, total: 2.23 s
Wall time: 2.23 s


---
## 모델 훈련

In [6]:
''' Prophet 예측을 위한 dataframe 포맷 제작 '''
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

In [7]:
@ray.remote
def pred_all_columns_chunk(plc_lst_partial,df_dic,fct_lst):
    res_list = []
    for i in plc_lst_partial:
        plc_df= df_dic[i]
        for j in fct_lst: # 피쳐 리스트
            fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
            fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
            
            if j in ['COD', 'SS',' T']:
                model = Prophet(
                    changepoint_prior_scale = 0.01,)
            else:
                 model = Prophet()
                    
            model.fit(fct_df)
            forecast = model.predict(future)
            forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
            forecast = forecast[['ds', 'yhat']]

            #for index, k in enumerate(list(forecast['ds'])):
            #    submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])

            res_list.append( (i,j,forecast) )
    return res_list

In [8]:
fct_lst_ray = ray.put(fct_lst)
df_dic_ray = ray.put(df_dic)

In [9]:
''' plc_lst 를 나눠서 병렬처리 '''
plc_splited = [list(x) for x in  list(np.array_split(plc_lst, 48 ))] # 48코어

In [ ]:
''' 주의 : 백그라운드에서 돌아가므로, 
cpu 로드율을 보고, 로드율이 100% 가까이 올라갔다가, 
아이들링 수치로 떨어졌을때 작업이 끝난거임. 
그전에는 다음 코드를 실행하지 말것. '''
task_list = [ pred_all_columns_chunk.remote(x,df_dic_ray,fct_lst_ray) for x in plc_splited ]

In [11]:
''' 결과 합치기 '''
results = ray.get(task_list)
results_combined = list(reduce(lambda f ,s : f +s ,results))

---
## TEST 데이터 포멧으로 바꾸기

In [12]:
%%time
''' 결과정리.'''
res_splited = [list(x) for x in  list(np.array_split(results_combined, 545 ))] # 48코어

for items in res_splited:
    df_lst = []
    for pred_info in items:
        comname = pred_info[0]
        col_name = pred_info[1]
        
        df_pred = pred_info[2]
        df_pred.columns = ['ds',col_name]
        df_lst.append(df_pred)
    df_combined = pd.concat(df_lst,axis = 1)
    df_sample = df_combined.loc[:, ~df_combined.columns.duplicated()]
    df_sample.index = df_sample.ds
    df_sample = df_sample.drop(['ds'],axis = 1)
    
    submit[comname] = df_sample.to_dict('index')

/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


CPU times: user 1.87 s, sys: 0 ns, total: 1.87 s
Wall time: 1.86 s


In [13]:
# 추론 파일 저장
with open('./sub/18_bs_fillna_cp0_01onlyCod_SS_T.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)